<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import spacy
import gensim.downloader as api
import pathlib
from sentence_splitter import SentenceSplitter, split_text_into_sentences

In [3]:
file_name = "Little_Red_Cap_ Jacob_and_Wilhelm_Grimm.txt"

# Read the text from the file
text = pathlib.Path(file_name).read_text(encoding='utf-8')

In [6]:
splitter = SentenceSplitter(language='en')
sentences = splitter.split(text)

In [16]:
long_sentences = []
for sentence in sentences:
    word_count = len(sentence.split())
    if word_count > 4:
        long_sentences.append(sentence)

In [19]:
df = pd.DataFrame(long_sentences, columns=['Sentence'])

In [28]:
df

,Sentence
0,Once upon a time there was a sweet little girl.
1,"Everyone who saw her liked her, but most of al..."
2,Once she gave her a little cap made of red vel...
3,"Because it suited her so well, and she wanted ..."
4,"One day her mother said to her, ""Come Little R..."
...,...
100,Marie Hassenpflug (1788-1856) provided them wi...
101,The German title of this tale is Rotkäppchen (...
102,Link to an English translation of the Grimms' ...
103,Link to the German text of the Grimms' final v...
